In [1]:
import numpy as np
import torch
import torch.optim as optim
from dataloaders import GetMultiStepDataLoaders
import model
from train import train_model

In [2]:
batch_size = 32
output_folder = "./output_cogging/"
save_results = True
use_GPU = True
latent_size = 128
epochs = 201
max_rollout_horizon = 10

In [3]:
data  = np.load('../data/RL_random_cogging_processed.npz')
states = data['coords']
steps = data['steps']
positions = data['positions']
rotations = data['rotations']

In [4]:
actions = np.hstack((steps, positions, rotations))

In [5]:
print(states.shape)
print(actions.shape)

(29800, 2048, 3)
(29800, 8)


In [6]:
point_size = states.shape[1]

In [7]:
series_length = 149
states_list, actions_list = [], []
for i in range(0, len(states), series_length):
    if i + series_length <= len(states):
        states_list.append(states[i:i+series_length])
        actions_list.append(actions[i:i+series_length])

In [8]:
train_loader, test_loader = GetMultiStepDataLoaders(states_list, actions_list, batch_size, rollout_horizon=max_rollout_horizon)

In [9]:
net = model.PCTransitionModel(point_size, latent_size)

if(use_GPU):
    device = torch.device("cuda:0")
    if torch.cuda.device_count() > 1: # if there are multiple GPUs use all
        net = torch.nn.DataParallel(net)
else:
    device = torch.device("cpu")

net = net.to(device)

In [10]:
optimizer = optim.Adam(net.parameters(), lr=0.0005)

In [11]:
train_model(train_loader, test_loader, net, epochs, max_rollout_horizon, optimizer, device, save_results, output_folder)

In [12]:
torch.save(net.state_dict(), './transition_model.pth')